## Pour rouler le code, il faut d'abord changer le path pour load les fichiers de données

In [0]:
path = '/content/drive/My Drive/Demo ML/Devoir4/'

In [0]:
# http://pytorch.org/
from os.path import exists
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.1-{platform}-linux_x86_64.whl torchvision

In [29]:
import os
import pickle
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data.sampler import SubsetRandomSampler
import numpy as np
from torch.utils.data import DataLoader, Dataset
from torch.optim import Adam

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import time

import matplotlib.pyplot as plt
%matplotlib inline

print(f"Your version of Pytorch is {torch.__version__}. You should use a version >0.4.")

Your version of Pytorch is 0.4.1. You should use a version >0.4.


In [42]:
# If a GPU is available, use it
# Pytorch uses an elegant way to keep the code device agnostic
if torch.cuda.is_available():
    device = torch.device("cuda")
    use_cuda = True
else:
    device = torch.device("cpu")
    use_cuda = False
    
print(device)

from google.colab import drive
drive.mount('/content/drive/')

cuda
Mounted at /content/drive/


## Data / Données

In [0]:
# load data
train_images = np.load(path + 'train_images.npy', encoding='latin1')#('data/train_images.npy', encoding='latin1')
x_train = np.asarray([x for x in train_images[:, 1]])
train_labels = np.genfromtxt(path + 'train_labels.csv', names=True, delimiter=',', dtype=[('Id', 'i8'), ('Category', '<S12')])
y_train = np.asarray([x[1].decode('utf-8') for x in train_labels])

test_images = np.load(path + 'test_images.npy', encoding='latin1')
x_test = np.asarray([x for x in test_images[:, 1]])


In [0]:
class Dataset(Dataset):
    def __init__(self, x, y, transform = None):
        self.X = x.reshape(-1, 1, 32, 32).astype(np.float)#/255
        self.Y = np.array(y).astype(np.float)
        self.transform = transform
            
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        item = self.X[idx]
        label = self.Y[idx]
        
        if self.transform:
            item = self.transform(item)

        return (item, label)

In [0]:
# clean image noise
class block:
  
    def __init__(self, d, x, y):
      queue = [(x,y)]
      self.visited = []
      est = x
      west = x
      north = y
      south = y
      flag = np.zeros((d.shape[0],d.shape[1]))

      while queue:
        
#           print("queue is", queue)
          s = queue.pop(0)
          self.visited.append(s)
          flag[s[0]][s[1]] = 1
          
          if s[0]+1 < 100 and d[s[0]+1][s[1]] > 0 and not flag[s[0]+1][s[1]]:
            queue.append((s[0]+1,s[1]))
            flag[s[0]+1][s[1]] = 1
            if s[0]+1 > west:
              west = s[0]+1

          if s[1]+1 < 100 and d[s[0]][s[1]+1] > 0 and not flag[s[0]][s[1]+1]:
            queue.append((s[0],s[1]+1))
            flag[s[0]][s[1]+1] = 1
            if s[1]+1 > south:
              south = s[1]+1

          if s[0]-1 > 0 and d[s[0]-1][s[1]] > 0 and not flag[s[0]-1][s[1]]:
            queue.append((s[0]-1,s[1]))
            flag[s[0]-1][s[1]] = 1
            if s[0]-1 < est:
              est = s[0]-1

          if s[1]-1 > 0 and d[s[0]][s[1]-1] > 0 and not flag[s[0]][s[1]-1]:
            queue.append((s[0],s[1]-1))
            flag[s[0]][s[1]-1] = 1
            if s[1]-1 < north:
              north = s[1]-1

          if s[0]+1 < 100 and s[1]+1 < 100 and d[s[0]+1][s[1]+1] > 0 and not flag[s[0]+1][s[1]+1]:
            queue.append((s[0]+1,s[1]+1))
            flag[s[0]+1][s[1]+1] = 1
            if s[0]+1 > west:
              west = s[0]+1
            if s[1]+1 > south:
              south = s[1]+1

          if s[0]-1 > 0 and s[1]-1 > 0 and d[s[0]-1][s[1]-1] > 0 and not flag[s[0]-1][s[1]-1]:
            queue.append((s[0]-1,s[1]-1))
            flag[s[0]-1][s[1]-1] = 1
            if s[0]-1 < est:
              est = s[0]-1
            if s[1]-1 < north:
              north = s[1]-1

          if s[0]+1 < 100 and s[1]-1 > 0 and d[s[0]+1][s[1]-1] > 0 and not flag[s[0]+1][s[1]-1]:
            queue.append((s[0]+1,s[1]-1))
            flag[s[0]+1][s[1]-1] = 1
            if s[0]+1 > west:
              west = s[0]+1
            if s[1]-1 < north:
              north = s[1]-1

          if s[0]-1 > 0 and s[1]+1 < 100 and d[s[0]-1][s[1]+1] > 0 and not flag[s[0]-1][s[1]+1]:
            queue.append((s[0]-1,s[1]+1))
            flag[s[0]-1][s[1]+1] = 1
            if s[0]-1 < est:
              est = s[0]-1
            if s[1]+1 > south:
              south = s[1]+1

      self.size = (west-est+1, south-north+1)

def clean_noise(d, noise_size):
    negative = []
    for i in range(d.shape[0]):
      for j in range(d.shape[1]):
        if d[i][j] > 0:
          b = block(d,i,j)
          if b.size < (noise_size[0],noise_size[1]):
            for dots in b.visited:
              d[dots[0]][dots[1]] = 0 
          else:
            negative.append(b.visited)
            for dots in b.visited:
              d[dots[0]][dots[1]] = -d[dots[0]][dots[1]]
    
    biggest = 0
    for h in negative:
      if len(h) > biggest:
        biggest = len(h)
    
    for g in negative:
        if len(g) == biggest:
          for p in g:
            d[p[0]][p[1]] = -d[p[0]][p[1]]
        else:
          for p in g:
            d[p[0]][p[1]] = 0
    
    return d
  
  
#######################################################

def centralize(image, dimInitial, dimCentre):
#Find bounding box of the image
  col_sum = np.where(np.sum(image, axis = 0)>0)
  row_sum = np.where(np.sum(image, axis = 1)>0)
  
  if len(col_sum[0])== 0 and len(row_sum[0]) == 0 :
    return image[0:dimCentre, 0:dimCentre]
  
  y1 = row_sum[0][0]
  x1 = col_sum[0][0]
  
  padX, padY = 0, 0
  y2 = y1 + dimCentre
  x2 = x1 + dimCentre
  
  if x2 > dimInitial:
    padX = x2 - dimInitial
    x2 = dimInitial
    
  if y2 > dimInitial:
    padY = y2 - dimInitial
    y2 = dimInitial
    
  img = image[y1:y2, x1:x2]
  
  if padX > 0:
    pad = np.zeros((y2-y1, padX))
    img = np.concatenate((img, pad), axis = 1)
  
  if padY > 0:
    pad = np.zeros((padY, dimCentre))
    img = np.concatenate((img, pad), axis = 0)
    
  return img


In [0]:
num_train = x_train.shape[0]
num_classes = 31
dimCentre = 32
num_features = dimCentre*dimCentre


#Pretraiter les images
trainProcessed = np.zeros((num_train, num_features))
for i in range(num_train):
  image = x_train[i].reshape(100,100)
  clean_noise(image,noise_size=(9,9))
  image = centralize(image, 100, dimCentre)
  trainProcessed[i] = image.reshape(1, num_features)


# for i in range(10):
#   plt.imshow(trainProcessed[i].reshape(32,32))
#   plt.show()

In [0]:
#Creer des images transformé de images train
import cv2 as cv

def translation(img, x, y):
  rows,cols = img.shape
  M = np.float32([[1,0,x],[0,1,y]])
  return cv.warpAffine(img,M,(cols,rows))

def rotation(img, angle):
  rows,cols = img.shape
  # cols-1 and rows-1 are the coordinate limits.
  M = cv.getRotationMatrix2D(((cols-1)/2.0,(rows-1)/2.0),angle,1)
  return cv.warpAffine(img,M,(cols,rows))

def transformerDataset(dataSet, d = 32):
  
  n = dataSet.shape[0]
  
  trainTransforme = np.zeros((n, d*d))
  
  for i in range(n//4):
    img = dataSet[i].reshape(d,d)
    img = translation(img,3,3) 
    trainTransforme[i] = img.reshape(1, d*d)
  
  for i in range(n//4, 2*(n//4)):
    img = dataSet[i].reshape(d,d)
    img = translation(img,3,3)
    img = rotation(img,20) 
    trainTransforme[i] = img.reshape(1, d*d)

  for i in range(2*(n//4), 3*(n//4)):
    img = dataSet[i].reshape(d,d)
    img = rotation(img,340) 
    trainTransforme[i] = img.reshape(1, d*d)
    
  for i in range(3*(n//4), n):
    img = dataSet[i].reshape(d,d)
    img = rotation(img,320) 
    trainTransforme[i] = img.reshape(1, d*d)
    
  return trainTransforme


imTrans = transformerDataset(trainProcessed)


In [0]:

def transformerDatasetII(dataSet, d=32):
  
  n = dataSet.shape[0]
  angle = 18
  x, y= 2, 2
  
  trainTransforme = np.zeros((n, d*d))

  for i in range(n):
    img = dataSet[i].reshape(d,d)
    rotOrTran = np.random.randint(3)
    if rotOrTran==0:
      c = np.random.randint(10)
      img = translation(img,3,3)
      img = rotation(img,angle-c) 
    elif rotOrTran==1:
      c = np.random.randint(10)
      img = rotation(img,360-angle-c) 
    else:
      c = np.random.randint(5)
      img = translation(img,x+c,y+c)

    trainTransforme[i] = img.reshape(1, d*d)
    
  return trainTransforme


imTrans2 = transformerDatasetII(dataSet=trainProcessed)

# for i in range(9000,9020):
#   plt.imshow(imTrans2[i].reshape(32,32))
#   plt.show()
#   plt.imshow(trainProcessed[i].reshape(32,32))
#   plt.show()
  

In [0]:


le = LabelEncoder()
y_train_enc = le.fit_transform(y_train) 




In [0]:
batch_size = 32

# concatener les 2 dataset
newTrain = np.concatenate((trainProcessed, imTrans), axis=0)
newTrain = np.concatenate((newTrain, imTrans2), axis=0)

newLabel = np.concatenate((y_train_enc, y_train_enc), axis=0)
newLabel = np.concatenate((newLabel, y_train_enc), axis=0)


xTrain, xTest, yTrain, yTest = train_test_split(newTrain, newLabel, test_size = 0.2, random_state = 1)

trainBatches = Dataset(xTrain, yTrain)
testBatches= Dataset(xTest, yTest)


train_loader = torch.utils.data.DataLoader(dataset = trainBatches, batch_size = batch_size, shuffle = False)
test_loader = torch.utils.data.DataLoader(dataset = testBatches, batch_size = batch_size, shuffle = False)


In [0]:
f = open("/content/drive/My Drive/Demo ML/Devoir4/newdata.csv", "w")
# f.write("Id,Category\n")
for inputs in newTrain:
  f.write(str(inputs)+"\n")
  
f.close()


In [0]:

# for i in range(19991, 20000):
#   print(le.inverse_transform(newLabel[i]))
#   plt.imshow(newTrain[i].reshape(32,32))
#   plt.show()
  

## Models / Modèles
You can find below a basic CNN. You will have to define your own model a bit later. First let's try to train this one!

Vous avez ci-dessous un CNN élémentaire. Vous devrez définir votre propre modèle un peu plus tard. Pour l'instant essayons déjà d'entraîner celui-ci.

In [0]:
class BasicNet(nn.Module):
    """Affordable convolutions for the people."""
    
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 32, 5, padding=2)
        self.conv2 = nn.Conv2d(32, 64, 5, padding=2)
        self.bn = nn.BatchNorm2d(64)
        self.fc = nn.Linear(64*8*8, 31)

    def forward(self, xin):
        # x is [batch_size, channels, heigth, width] = [bs, 1, 28, 28]
        x = xin + F.relu(self.conv1(xin))
        x = F.max_pool2d(x, 2) 
        # x is [bs, 32, 14, 14]
        x = F.relu(self.bn(self.conv2(x)))
        x = F.max_pool2d(x, 2) 
        # x is [bs, 64, 7, 7]
        x = x.view(-1, 64*8*8 ) # flatten
        x = F.relu(self.fc(x))
        return x
    

## Training / Entraînement

You have to define general training and testing loops that can be applied to any pytorch module.

Vous devez définir des boucles d'entraînement et de test générales qui s'appliquent à n'importe quel module pytorch.

In [0]:
# This parameter influences optimization
batch_size = 32
# This is just for evaluation, we want is as big as the GPU can support
batch_size_eval = 512


# Surrogate loss used for training
loss_fn = nn.CrossEntropyLoss()
test_loss_fn = nn.CrossEntropyLoss(reduction='sum')

# spot to save your learning curves, and potentially checkpoint your models
savedir = 'results'
if not os.path.exists(savedir):
    os.makedirs(savedir)

In [0]:
def train(model,train_loader, optimizer, epoch ):
    """Perform one epoch of training."""
    model.train()
    
    for batch_idx, (inputs, target) in enumerate(train_loader):
        inputs, target = inputs.float().to(device), target.long().to(device)
        
        # Let them code what's here
        optimizer.zero_grad()
        output = model(inputs)
        loss = loss_fn(output, target)
        loss.backward()
        optimizer.step()
        ###
        
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(inputs), len(train_loader) *len(inputs) ,
                100. * batch_idx / len(train_loader), loss.item()))
            

In [0]:
def test(model, test_loader):
    """Evaluate the model by doing one pass over a dataset"""
    model.eval()
    
    test_loss = 0
    correct = 0
    test_size = 0
    with torch.no_grad():
        for inputs, target in test_loader:
            inputs, target = inputs.float().to(device), target.long().to(device)
            
            # let them code what's here
            output = model(inputs)
            test_size += len(inputs)
            test_loss += test_loss_fn(output, target).item() # sum up batch loss
            pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= test_size
    accuracy = correct / test_size
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, test_size,
        100. * accuracy))
    
    return test_loss, accuracy

In [0]:
# model = BasicNet().to(device)

# lr = 0.0005
# optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9)

# results = {'name':'basic', 'lr': lr, 'loss': [], 'accuracy':[]}
# savefile = os.path.join(savedir, results['name']+str(results['lr'])+'.pkl' )

# for epoch in range(1, 100):
#     train(model, train_loader, optimizer, epoch)
#     loss, acc = test(model, test_loader)
    
#     # save results every epoch
#     results['loss'].append(loss)
#     results['accuracy'].append(acc)
#     with open(savefile, 'wb') as fout:
#         pickle.dump(results, fout)

We have just applied our basic model on scratch_loader. A decent convnet with good parameters should be able to overfit this data easily.
What happened ? What do you conclude ?

On viens d'entraîner notre convnet de base sur scratch_loader. Un modèle décent devrait être capable de mémoriser ce dataset aisément. Que s'est-il passé? Qu'en concluez vous?


## Build your model

It's time to implement your own models to get the best clasification performance on MNIST.
First, try to overfit scratch_loader. Once you succeed, replace it by the real loaders and classify these digits!
You may consider the following ideas, ranked by relevance:

* batch norm
* more layers
* skip connections
* dropout (on the high level features)
* data augmentation with `transforms.RandomRotation`or `transforms.RandomAffine` at the dataset creation time

If you need some help to understand padding, stride etc..., [here](https://github.com/vdumoulin/conv_arithmetic) is very good resource from a Mila alumni.

You can use the cell below to compare the learning curves of your models. Don't forget to change the `'name'`value in the `results`dictionary between each try.

## Fabriquez votre modèle

Vous devez maintenant implémenter votre propre modèle. Essayez d'abord de mémoriser scratch_loader. Ensuite essayez de bien classifier MNIST. Vous pouvez considérer les idées suivantes classées selon la préférence de l'auteur ce ces lignes:

* normalisation de lot 
* réseau plus profond
* connections sautées
* abandon de neurones
* augmentation de données avec les méthodes `transforms.RandomRotation`ou `transforms.RandomAffine`  lors de la création du jeux de données.

Si vous ne connaissez pas ces notions, c'est normal. Regardez la version anglaise. 
Si vous avez voulez mieux comprendre le fonctionnement des couches convolutionnelles, regardez [ces merveilleux gifs](https://github.com/vdumoulin/conv_arithmetic) réalisés par un ancien du Mila.

Vous pouvez utiliser la cellule ci-dessous pour comparer vos courbes d'apprentissages entre elles. N'oubliez pas de changer  la valeur `'name'` dans le dictionnaire `results`  entre chaque essais.

In [0]:
# # PLOTTING
# fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 4))

# for filename in os.listdir(savedir):
#     if filename.endswith('.pkl'):
#         with open(os.path.join(savedir, filename),'rb') as fin:
#             results = pickle.load(fin)
#             ax1.plot(results['loss'])
#             ax1.set_ylabel('cross entropy')
#             ax1.set_xlabel('epochs')
            
#             ax2.plot(results['accuracy'], label = filename[:-4])
#             ax2.set_ylabel('accuracy')
#             ax2.set_xlabel('epochs')
            
# plt.legend()

In [0]:
class HeavyNet(nn.Module):
    """A medium sized network that performs very well on MNIST."""
    
    def __init__(self):
        super().__init__()
        # conv block 1
        self.conv1 = nn.Conv2d(1, batch_size, 3, padding=1)
        self.bn1 = nn.BatchNorm2d(batch_size)
        self.conv2 = nn.Conv2d(32, 32, 3, padding=1)
        self.bn2 = nn.BatchNorm2d(32)
        
        # conv block 2
        self.conv3 = nn.Conv2d(batch_size, 64, 3, padding=1)
        self.bn3 = nn.BatchNorm2d(64)
        self.conv4 = nn.Conv2d(64, 64, 3, padding=1)
        self.bn4 = nn.BatchNorm2d(64)
        
        # fully connected layers
        self.fc1 = nn.Linear(64*8*8, 512)
        self.bn5 = nn.BatchNorm1d(512)
        self.dropout = nn.Dropout(0.2)
        self.fc2 = nn.Linear(512, 31)
        
    def forward(self, x):
        # x is [batch_size, channels, heigth, width] = [bs, 1, 32, 32]
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.max_pool2d(x, 2) # x is [bs, 32, 16, 16]
        
        x = F.relu(self.bn3(self.conv3(x)))
        x = F.relu(self.bn4(self.conv4(x)))
        x = F.max_pool2d(x, 2) # x is [bs, 64, 8, 8]
        
        x = x.view(x.size(0), -1) # flatten
        
        x = F.relu(self.bn5(self.fc1(x)))
        x = self.dropout(x)
        x = self.fc2(x)
        
        return x
    

In [20]:
# This parameter influences optimization
batch_size = 32
# This is just for evaluation, we want is as big as the GPU can support
batch_size_eval = 512

# TRAINING
model = HeavyNet().to(device)

lr = 0.005
optimizer = optim.Adam(model.parameters(), lr=lr)

# results = {'name':'awesome', 'lr': lr, 'loss': [], 'accuracy':[]}
# savefile = os.path.join(savedir, results['name']+str(results['lr'])+'.pkl' )

for epoch in range(1, 150):
    train(model, train_loader, optimizer, epoch)
    loss, acc = test(model, test_loader)
    
#     # save results
#     results['loss'].append(loss)
#     results['accuracy'].append(acc)
#     with open(savefile, 'wb') as fout:
#         pickle.dump(results, fout)

Train Epoch: 1 [0/24000 (0%)]	Loss: 3.385876
Train Epoch: 1 [320/24000 (1%)]	Loss: 2.976355
Train Epoch: 1 [640/24000 (3%)]	Loss: 2.769938
Train Epoch: 1 [960/24000 (4%)]	Loss: 2.458241
Train Epoch: 1 [1280/24000 (5%)]	Loss: 2.576741
Train Epoch: 1 [1600/24000 (7%)]	Loss: 2.450269
Train Epoch: 1 [1920/24000 (8%)]	Loss: 2.922332
Train Epoch: 1 [2240/24000 (9%)]	Loss: 2.322709
Train Epoch: 1 [2560/24000 (11%)]	Loss: 2.401452
Train Epoch: 1 [2880/24000 (12%)]	Loss: 1.691225
Train Epoch: 1 [3200/24000 (13%)]	Loss: 2.273591
Train Epoch: 1 [3520/24000 (15%)]	Loss: 1.971038
Train Epoch: 1 [3840/24000 (16%)]	Loss: 2.368343
Train Epoch: 1 [4160/24000 (17%)]	Loss: 2.365314
Train Epoch: 1 [4480/24000 (19%)]	Loss: 2.482097
Train Epoch: 1 [4800/24000 (20%)]	Loss: 1.872742
Train Epoch: 1 [5120/24000 (21%)]	Loss: 2.378906
Train Epoch: 1 [5440/24000 (23%)]	Loss: 1.733241
Train Epoch: 1 [5760/24000 (24%)]	Loss: 2.017887
Train Epoch: 1 [6080/24000 (25%)]	Loss: 2.527558
Train Epoch: 1 [6400/24000 (27%)]	

In [0]:
# class AwesomeNet(nn.Module):
#     def __init__(self):
#         super().__init__()
#         # conv block 1
#         self.conv1 = nn.Conv2d(1, batch_size, 3, padding=1)
#         self.bn1 = nn.BatchNorm2d(batch_size)
#         self.conv2 = nn.Conv2d(32, 32, 2)
#         self.bn2 = nn.BatchNorm2d(32)
        
#         # conv block 2
#         self.conv3 = nn.Conv2d(batch_size, 64, 3, padding=1)
#         self.bn3 = nn.BatchNorm2d(64)
#         self.conv4 = nn.Conv2d(64, 64, 3, padding=1)
#         self.bn4 = nn.BatchNorm2d(64)
        
#         # fully connected layers
#         self.fc1 = nn.Linear(64*8*8, 512)
#         self.bn5 = nn.BatchNorm1d(512)
#         self.dropout = nn.Dropout(0.2)
#         self.fc2 = nn.Linear(512, 31)
        
#     def forward(self, x):
#         # x is [batch_size, channels, heigth, width] = [bs, 1, 32, 32]
#         x = F.relu(self.bn1(self.conv1(x)))
#         x = F.relu(self.bn2(self.conv2(x)))
#         x = F.max_pool2d(x, 2) # x is [bs, 32, 16, 16]
        
#         x = F.relu(self.bn3(self.conv3(x)))
#         x = F.relu(self.bn4(self.conv4(x)))
#         x = F.max_pool2d(x, 2) # x is [bs, 64, 8, 8]
        
#         x = x.view(x.size(0), -1) # flatten
        
#         x = F.relu(self.bn5(self.fc1(x)))
#         x = self.dropout(x)
#         x = self.fc2(x)
        
#         return x
    
    

In [0]:
# # This parameter influences optimization
# batch_size = 32
# # This is just for evaluation, we want is as big as the GPU can support
# batch_size_eval = 512

# # TRAINING
# model = AwesomeNet().to(device)

# lr = 0.005
# optimizer = optim.Adam(model.parameters())

# results = {'name':'awesome', 'lr': lr, 'loss': [], 'accuracy':[]}
# savefile = os.path.join(savedir, results['name']+str(results['lr'])+'.pkl' )

# for epoch in range(1, 70):
#     train(model, train_loader, optimizer, epoch)
#     loss, acc = test(model, test_loader)
    
#     # save results
#     results['loss'].append(loss)
#     results['accuracy'].append(acc)
#     with open(savefile, 'wb') as fout:
#         pickle.dump(results, fout)

If you want to learn more about Pytorch, here is a very comprehensive [tutorial](https://nbviewer.jupyter.org/github/ds4dm/tipsntricks/blob/master/pytorch/tutorial.ipynb) made by Mila for Mila. You are encouraged to look at it **after** this lab session.

In [0]:
num_test = x_test.shape[0]
#Traiter les images à prédire
testProcessed = np.zeros((num_test, num_features))
for i in range(num_test):
  image = x_test[i].reshape(100,100)
  clean_noise(image,noise_size=(9,9))
  #image = rescale(image, 1.0 / 2.0)
  image = centralize(image, 100, dimCentre)
  testProcessed[i] = image.reshape(1, num_features)
  
# testProcessed = torch.from_numpy(testProcessed).float().to(device)

In [0]:
test_dataset = Dataset(testProcessed, np.zeros((x_test.shape[0], 1)))

test_loader2 = torch.utils.data.DataLoader(dataset = test_dataset, batch_size = batch_size, shuffle = False)

# test_loader = torch.utils.data.DataLoader(dataset = testProcessed, batch_size = batch_size, shuffle = False)

In [40]:
model.eval()
p = []

for images, labels in test_loader2:
  images = images.type(torch.cuda.FloatTensor)#(inputs.type(torch.cuda.FloatTensor))#
  outputs = model(images)
  _, predicted = torch.max(outputs.data, 1)
  for i in predicted:
      p.append(i)
#   print("finished")
  

predictions = le.inverse_transform(p)
# prediction = model(testProcessed)
# prediction = prediction.max(1, keepdim=False)[1]





/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


['pineapple' 'pear' 'pool' ... 'skateboard' 'pineapple' 'mug']


In [0]:
f = open(path + "myPrediction.csv", "w")
f.write("Id,Category\n")

for i,c in enumerate(predictions):
  f.write(str(i)+","+c+"\n")
  
f.close()